<a href="https://colab.research.google.com/github/elly-4/aegle/blob/master/Neurological_CP_of_ColabProj0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install and Import dependencies**

In [ ]:
!pip install mediapipe opencv-python


In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import mediapipe as mp
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import PIL
import io
import html
import time
from google.colab.patches import cv2_imshow


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic


**Get Realtime Webcam Feed**

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

## **Xtremely long video capture code**

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label_html, bbox ):
  data = eval_js('stream_frame("{}", "{}")'.format(label_html, bbox))
  return data

## **Calling the video function**

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
          break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
     
      cv2_imshow(image)

In [ ]:

results.face_landmarks.landmark[0].visibility

## 2. **Capture Landmarks & Export to CSV** 

In [ ]:
import csv
import os
import numpy as np


In [ ]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

### **landmarks**

In [ ]:
landmarks

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### **ClassName Set**

In [ ]:

class_name = "HipAbductors3"

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
       break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
      # 4. Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
      # Export coordinates
      try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
      except:
            pass

    
      cv2_imshow(image)
 

## **3. Train Custom Model Using Scikit Learn**

### **3.1 Read in Collected Data and Process**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['class']=='FacialAppearanceScore0']

In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
y_test

### **3.2 Train Machine Learning Classification Model**

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:

fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

### **3.3 Evaluate and Serialize Model¶**

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))


In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
y_test

In [ ]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## **4. Make Detections with Model**

In [ ]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model

In [ ]:
# start streaming video from webcam
video_stream()

# # label for video
label_html = 'Capturing...' 
# initialze bounding box to empty
bbox = ''
count = 0 
# image = '' 

#Intiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while True:
      js_reply = video_frame(label_html, bbox )
    
      if not js_reply:
       break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])

      #Recolour feed
      image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      #Make Detections
      results = holistic.process(image)
      print(results.face_landmarks)

      #face_Landmarks, pose_Landmarks, left_Landmarks, right_Landmarks

      #Recolor image back ot BGR for Rendering 
      image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

      #Draw Face landmarks 
      mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)

      #Right hand 
      mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      #Left hand
      mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

      # Pose Detections
      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
       
      # Export coordinates
      try:
          # Extract Pose landmarks
          pose = results.pose_landmarks.landmark
          pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
          
          # Extract Face landmarks
          face = results.face_landmarks.landmark
          face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
          
          # Concate rows
          row = pose_row+face_row
          
#             # Append class name 
#             row.insert(0, class_name)
          
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

          # Make Detections
          X = pd.DataFrame([row])
          body_language_class = model.predict(X)[0]
          body_language_prob = model.predict_proba(X)[0]
          print(body_language_class, body_language_prob)
          
          # Grab ear coords
          coords = tuple(np.multiply(
                          np.array(
                              (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                      , [640,480]).astype(int))
          
          cv2.rectangle(image, 
                        (coords[0], coords[1]+5), 
                        (coords[0]+len(body_language_class)*20, coords[1]-30), 
                        (245, 117, 16), -1)
          cv2.putText(image, body_language_class, coords, 
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
          # Get status box
          cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
          
          # Display Class
          cv2.putText(image, 'CLASS'
                      , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
          cv2.putText(image, body_language_class.split(' ')[0]
                      , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
          # Display Probability
          cv2.putText(image, 'PROB'
                      , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
          cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                      , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
      except:
          pass
                        
    

           

    
      cv2_imshow(image)

In [ ]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))